<a href="https://colab.research.google.com/github/brenoakihiromorimoto/portf-lio/blob/main/%237DaysOfCode/dia_4_%F0%9F%91%A9%F0%9F%8F%BD%E2%80%8D%F0%9F%92%BB_Sistemas_de_recomenda%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#7DaysOfCode - Ciência de Dados 4/7: 👩🏽‍💻 Sistemas de recomendação
Olha só, você está na metade do #7DaysOfCode de Dados! Espero que esteja aproveitando muito e desenvolvendo novas habilidades.

Hoje vou começar falando sobre Machine Learning.

Este conceito tem sido utilizado para resolver diversos problemas em todos os setores da indústria.

Se você já usou o Google Maps, Spotify, Youtube, ou Amazon, é certo que já esbarrou em algum sistema utilizando aprendizado de máquina para melhorar sua experiência e criar novos produtos.

Uma das aplicações que mais interessam empresas de todos os setores são os sistemas de recomendação: aplicações que aprendem, a partir de dados históricos, como sugerir os melhores itens para clientes a fim de maximizar alguma métrica de negócio, como faturamento, taxa de conversão, dentre outras.

E é exatamente sobre isso que eu vou falar no desafio de hoje.

O MovieLens é um dataset clássico usado em problemas de sistema de recomendação. Ele é usado até em artigos científicos para validar novos tipos de algoritmos de recomendação.

Eu gosto desse conjunto de dados porque ele está muito próximo do mercado.

Afinal, quantas empresas não se beneficiariam de um sistema de recomendações?

Minha proposta para este desafio é que você crie um sistema de recomendações a partir do MovieLens.

Pense em como uma empresa como a Netflix utilizaria um sistema desse jeito. Você consegue criar um sistema que recomenda 5 filmes para o usuário com base em comportamentos passados?

# Importar Bibliotecas

In [88]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
!pip install surprise
from surprise import SVD, NormalPredictor, KNNBasic
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from google.colab import drive
import surprise
drive.mount('/gdrive')
%cd /gdrive
import joblib

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


# Importar datasets

In [12]:
filmes = pd.read_csv('/gdrive/MyDrive/Colab Notebooks/#7DaysOfCode/movies.csv', sep=',')
rating = pd.read_csv('/gdrive/MyDrive/Colab Notebooks/#7DaysOfCode/ratings.csv', sep=',')
tags = pd.read_csv('/gdrive/MyDrive/Colab Notebooks/#7DaysOfCode/tags.csv', sep=',')

# Preparar o dataset para fazer validação cruzada.

In [14]:
rating.head(3)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182


In [19]:
reader = Reader(rating_scale=(1, 5))
rating_reader = Dataset.load_from_df(rating[['userId', 'movieId', 'rating']], reader)

# Validação cruzada dos modelos.

In [77]:
algoritmos = [SVD(), NormalPredictor(), KNNBasic()]
total_results = pd.DataFrame()
for algoritmo in algoritmos:
  results = cross_validate(algoritmo, rating_reader, measures=['RMSE'], cv=5)
  row_results = pd.DataFrame(pd.DataFrame.from_dict(results).mean(axis=0)).T.rename(index={0:str(algoritmo).split('.')[-1].split(' ')[0]})
  total_results = total_results.append(row_results)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


# Resultados

In [216]:
total_results.sort_values(by='test_rmse')

,test_rmse,fit_time,test_time
SVD,0.897294,6.051691,0.248913
KNNBasic,0.967685,0.210660,1.807202
NormalPredictor,1.445094,0.203097,0.197427


* O NormalPredictor é um algoritmo que prevê a avaliação dos filmes de forma aleatória, isso foi considerado a baseline.
* O SVD teve o RMSE mais baixo que a baseline e o KNNBasic, a diferença média entre o alvo e a previsão é de 0.89 estrelas.

# Justificativa das métricas.

* A métrica usada foi o rmse, porque o alvo foi prever a avaliação dos filmes dada por um usuário.

# Treinar modelo.

In [85]:
trainset = rating_reader.build_full_trainset()

In [109]:
algo = SVD().fit(trainset)

# Lista de recomendação de filmes por usuário

* Recomendação de filmes por usuário.

In [218]:
def top_5_filmes(user_id, algoritmo):
  all_ratings = pd.DataFrame()
  for movie_id in rating.movieId.unique().tolist():
    row = {'rating': algoritmo.predict(user_id, movie_id).est,
          'movie_id': str(movie_id)}
    all_ratings = all_ratings.append(row, ignore_index=True)
  else:
    top_5_rating = all_ratings.sort_values(by='rating', ascending=False).iloc[:5]
    return top_5_rating.movie_id.tolist()

In [219]:
top_5_filmes = top_5_filmes(1, algo)

In [220]:
top_5_filmes

['527', '1228', '7502', '1172', '1252']

# Salvar o modelo em .sav no Google Drive

In [110]:
filename = '/gdrive/MyDrive/Colab Notebooks/#7DaysOfCode/modelo_finalizado.sav'
joblib.dump(algo, filename)

['/gdrive/MyDrive/Colab Notebooks/#7DaysOfCode/modelo_finalizado.sav']

In [111]:
loaded_model = joblib.load(filename)

In [221]:
loaded_model.predict('1', '527')

Prediction(uid='1', iid='527', r_ui=None, est=3.543608255669773, details={'was_impossible': False})